### IE6600 Computation and Visualization for Analytics
#### Spring 2025

### Geospatial Data Analysis and Interactive Visualization

#### Dataset Selection and Confirmation

Link to the dataset: https://catalog.data.gov/dataset/solar-footprints-in-california-6251a

#### Data Acquisition and Inspection

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("Solar_Footprints.csv")

# Display the first 5 rows
df.head()


In [ ]:
# Column names and data types
print("\nData Types and Structure:")
print(df.info())

#### Data Cleaning and Preparation

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

In [ ]:
# Drop rows with any missing values
df.dropna(inplace=True)

# Check again for missing values
print("\nMissing values after dropping rows:")
print(df.isnull().sum())

In [ ]:
# Check and remove duplicate rows
initial_count = df.shape[0]
df.drop_duplicates(inplace=True)
final_count = df.shape[0]

print(f"\nDuplicates removed: {initial_count - final_count}")
print(f"Final dataset shape: {df.shape}")

In [ ]:
# Threshold for "low" number of unique values (adjustable)
threshold = 10

# Find columns with fewer than `threshold` unique values
low_card_cols = [col for col in df.columns if df[col].nunique() <= threshold]

print(f"Columns with <= {threshold} unique values:")
print(low_card_cols)

# Show the unique values for each of those columns
for col in low_card_cols:
    print(f"\nColumn: {col}")
    print("Unique values:", df[col].unique())
    print("Value counts:")
    print(df[col].value_counts())


In [ ]:
import numpy as np

# Select only numeric columns
numeric_cols = df.select_dtypes(include=np.number).columns

# Dictionary to store outlier counts
outlier_summary = {}

# Iterate over numeric columns and flag outliers
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Create a new column indicating whether the value is an outlier
    outlier_col = f"{col}_is_outlier"
    df[outlier_col] = ((df[col] < lower_bound) | (df[col] > upper_bound))

    # Count outliers
    outlier_count = df[outlier_col].sum()
    outlier_summary[col] = outlier_count

# Display outlier counts per column
print("Outlier counts in numeric columns:")
for col, count in outlier_summary.items():
    print(f"{col}: {count} outliers")

# Optional: Remove outliers if needed
# df = df[~df[[f"{col}_is_outlier" for col in numeric_cols]].any(axis=1)]


#### Geospatial Data Analysis and Interactive Visualization

In [ ]:
import plotly.express as px

# Get top 10 counties by total acres
top_10_counties = (
    df.groupby('County')['Acres']
    .sum()
    .nlargest(10)
    .index
)

# Filter the DataFrame to include only top 10 counties
filtered_df = df[df['County'].isin(top_10_counties)]

# Plot the bar chart
fig = px.bar(
    filtered_df,
    x='County',
    color='Install Type',
    title='Top 10 Counties By Acres: Distribution of Install Types',
    labels={'Install Type': 'Type of Solar Installation'},
    barmode='group'
)

fig.show()
# Save the chart as an HTML file
fig.write_html("top_10_counties_install_type.html")


In [ ]:
# Scatter Plot: Acres vs. Distance to Substation (CAISO) 
import plotly.express as px

fig = px.scatter(
    df,
    x='Distance to Substation (Miles) CAISO',
    y='Acres',
    color='Install Type',
    title='Acres vs. Distance to CAISO Substation',
    labels={
        'Distance to Substation (Miles) CAISO': 'Distance to CAISO Substation (Miles)',
        'Acres': 'Footprint Size (Acres)',
        'Install Type': 'Type of Installation'
    },
    hover_data=['County', 'Combined Class']
)

fig.show()
fig.write_html("acres_vs_caiso_distance.html")

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
import plotly.express as px

# Step 2: Load GeoJSON
gdf = gpd.read_file("Solar_Footprints.geojson")

# Step 3: Merge on OBJECTID
merged = gdf.merge(df, on='OBJECTID')

# Step 3: Compute centroid in projected CRS for accurate center
projected = merged.to_crs(epsg=3857)
centroid = projected.geometry.centroid
center_x, center_y = centroid.x.mean(), centroid.y.mean()

# Convert back to EPSG:4326 for web plotting
merged = merged.to_crs(epsg=4326)

# Step 4: Plot using Plotly choropleth_mapbox
fig = px.choropleth_mapbox(
    merged,
    geojson=merged.geometry,
    locations=merged.index,
    color='Urban or Rural',
    hover_name='County',
    hover_data={'Acres_y': True, 'Urban or Rural': True},
    center={"lat": merged.geometry.centroid.y.mean(), "lon": merged.geometry.centroid.x.mean()},
    zoom=7,
    opacity=0.6,
    mapbox_style="carto-positron"
)

fig.update_layout(
    title="Solar Footprints by County (Rural vs Urban)",
    margin={"r":0,"t":40,"l":0,"b":0}
)

# Show the plot
fig.show()

# Save as HTML
fig.write_html("solar_footprints_map.html")


In [ ]:
merged = merged.to_crs(epsg=3857)
merged['centroid'] = merged.geometry.centroid

# Set centroid as the active geometry (this makes .x/.y work)
merged = merged.set_geometry('centroid').to_crs(epsg=4326)

# Extract lat/lon
merged['lon'] = merged.geometry.x
merged['lat'] = merged.geometry.y

# Create clustered scatter map (interactive!)
fig = px.scatter_mapbox(
    merged,
    lat='lat',
    lon='lon',
    color='Urban or Rural',
    hover_name='County',
    size='Acres_y',
    zoom=5,
    mapbox_style='carto-positron'
)

fig.update_layout(
    title='Clusters of Solar Footprints in California',
    margin={"r":0,"t":40,"l":0,"b":0}
)

fig.show()
fig.write_html("clusters_solar_footprints_map.html")

In [ ]:
import geopandas as gpd
import plotly.express as px

# Define proximity threshold (e.g., 20 miles)
threshold = 20

# Create boolean flags for each substation type being within threshold
df['close_100kV'] = df['Distance to Substation (Miles) GTET 100 Max Voltage'] <= threshold
df['close_200kV'] = df['Distance to Substation (Miles) GTET 200 Max Voltage'] <= threshold
df['close_CAISO'] = df['Distance to Substation (Miles) CAISO'] <= threshold

# Count how many substations are close for each object
df['substation_count_nearby'] = df[['close_100kV', 'close_200kV', 'close_CAISO']].sum(axis=1)

# Get top 30 entries with most nearby substations
top_30 = df.sort_values(by='substation_count_nearby', ascending=False).head(30)

# Merge back with spatial data
top_30['OBJECTID'] = top_30['OBJECTID'].astype(int)
gdf['OBJECTID'] = gdf['OBJECTID'].astype(int)
merged_top = gdf.merge(top_30, on='OBJECTID')

# Convert to centroid points
merged_top = merged_top.to_crs(epsg=3857)
merged_top['centroid'] = merged_top.geometry.centroid
merged_top = merged_top.set_geometry('centroid').to_crs(epsg=4326)
merged_top['lon'] = merged_top.geometry.x
merged_top['lat'] = merged_top.geometry.y

# Create the scatter mapbox plot
fig = px.scatter_mapbox(
    merged_top,
    lat='lat',
    lon='lon',
    size='substation_count_nearby',
    color='Urban or Rural',
    hover_name='County',
    hover_data={
        'substation_count_nearby': True,
        'Acres_y': True,
        'Urban or Rural': True
    },
    zoom=6,
    size_max=10,
    title='Top 30 Solar Footprints with Most Substations Nearby (Within 20 Miles)',
    mapbox_style="carto-positron"
)

fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 0})
fig.show()

# Save as HTML
fig.write_html("top_30_substations_nearby_map.html")


In [ ]:
import plotly.express as px
import pandas as pd

# Step 1: Clean substation column for clarity
merged_top['Substation'] = merged_top['Substation Name GTET 100 Max Voltage']
# Add Substation column in df, not merged_top
df['Substation'] = df['Substation Name GTET 100 Max Voltage']
df['is_close_100kv'] = df['Distance to Substation (Miles) GTET 100 Max Voltage'] <= 10

filtered_df = df[df['is_close_100kv']]
grouped = filtered_df.groupby(['Substation', 'Install Type']).size().reset_index(name='count')


# Compute total counts per substation
top_20_subs = (
    grouped.groupby('Substation')['count']
    .sum()
    .sort_values(ascending=False)
    .head(20)
    .index
)

# Filter only those top 20 substations
grouped_top20 = grouped[grouped['Substation'].isin(top_20_subs)]

fig = px.bar(
    grouped_top20,
    x='Substation',
    y='count',
    color='Install Type',
    title='Top 20 Substations Closest to the Most Solar Footprints (Stacked by Install Type)',
    labels={'count': 'Number of Solar Footprints'},
    text='count'
)

fig.update_layout(
    barmode='stack',
    xaxis_title='Substation Name (GTET 100kV)',
    yaxis_title='Number of Nearby Solar Footprints',
    xaxis_tickangle=45,
    margin=dict(l=20, r=20, t=60, b=120)
)

fig.show()
fig.write_html("top_20_substations_stacked_install_type.html")

In [ ]:
import pandas as pd
import plotly.express as px

# Define threshold for "closeness" (e.g., 10 miles)
threshold = 10

# Create proximity flags
df['close_to_100kv'] = df['Distance to Substation (Miles) GTET 100 Max Voltage'] <= threshold
df['close_to_200kv'] = df['Distance to Substation (Miles) GTET 200 Max Voltage'] <= threshold
df['close_to_caiso']  = df['Distance to Substation (Miles) CAISO'] <= threshold

# ------------------------------
# 📊 GTET 100kV - Bar Chart
# ------------------------------
sub_100 = (
    df[df['close_to_100kv']]
    .groupby('Substation Name GTET 100 Max Voltage')
    .size()
    .sort_values(ascending=False)
    .head(20)
    .reset_index(name='Number of Solar Footprints')
)

fig1 = px.bar(
    sub_100,
    x='Substation Name GTET 100 Max Voltage',
    y='Number of Solar Footprints',
    title='Top 20 GTET 100kV Substations by Solar Footprints (within 10 miles)',
    labels={'Number of Solar Footprints': 'Number of Solar Footprints', 'Substation Name GTET 100 Max Voltage': 'Substation'},
    text='Number of Solar Footprints'
)
fig1.update_layout(xaxis_tickangle=45)
fig1.show()

# ------------------------------
# 🔘 GTET 200kV - Bubble Plot
# ------------------------------
sub_200 = (
    df[df['close_to_200kv']]
    .groupby('Substation Name GTET 200 Max Voltage')
    .size()
    .sort_values(ascending=False)
    .head(20)
    .reset_index(name='Number of Solar Footprints')
)

fig2 = px.scatter(
    sub_200,
    x='Substation Name GTET 200 Max Voltage',
    y='Number of Solar Footprints',
    size='Number of Solar Footprints',
    color='Number of Solar Footprints',
    title='Top 20 GTET 200kV Substations (Bubble Plot)',
    labels={'Substation Name GTET 200 Max Voltage': 'Substation'},
    hover_name='Substation Name GTET 200 Max Voltage',
)

fig2.update_traces(marker=dict(sizemode='area', sizeref=2.*max(sub_200['Number of Solar Footprints'])/100**2))
fig2.update_layout(xaxis_tickangle=45, margin=dict(t=60, b=120))
fig2.show()

# ------------------------------
# 🧮 CAISO - Treemap
# ------------------------------
power_caiso = (
    df[df['close_to_caiso']]
    .groupby('Substation CASIO Name')
    .size()
    .sort_values(ascending=False)
    .head(20)
    .reset_index(name='Number of Solar Footprints')
)

fig3 = px.treemap(
    power_caiso,
    path=['Substation CASIO Name'],
    values='Number of Solar Footprints',
    title='Top 20 CAISO Substations by Solar Footprints (Treemap)',
    color='Number of Solar Footprints',
    color_continuous_scale='Viridis'
)

fig3.show()
fig1.write_html("top_20_GTET_100kV_substations.html")
fig2.write_html("top_20_GTET_200kV_substations.html")
fig3.write_html("top_20_CAISO_substations_treemap.html")

In [ ]:
import plotly.express as px
import pandas as pd

# Step 1: Prepare a long-format dataframe
df_100 = df[['Acres', 'Distance to Substation (Miles) GTET 100 Max Voltage', 'Percentile (GTET 100 Max Voltage)']].copy()
df_100.columns = ['Acres', 'Distance', 'Percentile']
df_100['Substation Type'] = 'GTET 100kV'

df_200 = df[['Acres', 'Distance to Substation (Miles) GTET 200 Max Voltage', 'Percentile (GTET 200 Max Voltage)']].copy()
df_200.columns = ['Acres', 'Distance', 'Percentile']
df_200['Substation Type'] = 'GTET 200kV'

df_caiso = df[['Acres', 'Distance to Substation (Miles) CAISO', 'Percentile (CAISO)']].copy()
df_caiso.columns = ['Acres', 'Distance', 'Percentile']
df_caiso['Substation Type'] = 'CAISO'

# Combine all into one long dataframe
long_df = pd.concat([df_100, df_200, df_caiso])

# Step 2: Create scatter plot with facet per substation type
fig = px.scatter(
    long_df,
    x='Distance',
    y='Acres',
    color='Percentile',
    facet_col='Substation Type',
    title='Acres vs Distance to Substation, Colored by Percentile Bin',
    labels={'Distance': 'Distance to Substation (Miles)', 'Acres': 'Acres of Solar Footprint'},
    opacity=0.6,
    height=500
)

fig.update_layout(
    margin=dict(t=50, b=60, l=40, r=20),
    legend_title='Percentile Bin'
)
fig.show()
fig.write_html("acres_vs_distance_by_percentile_facet.html")

In [ ]:
import plotly.express as px
import geopandas as gpd

# Prepare centroids in lat/lon
gdf = gdf.to_crs(epsg=4326)
gdf['lon'] = gdf.geometry.centroid.x
gdf['lat'] = gdf.geometry.centroid.y

# Heatmap (weighted by Acres)
fig = px.density_mapbox(
    gdf,
    lat='lat',
    lon='lon',
    z='Acres',
    radius=20,
    center=dict(lat=gdf['lat'].mean(), lon=gdf['lon'].mean()),
    zoom=6,
    mapbox_style="carto-positron",
    title="Solar Acreage Density Heatmap"
)
fig.show()


In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.graph_objects as go
from shapely.geometry import Point, LineString

# 1. Simplify and reproject GeoDataFrame to EPSG:4326
gdf = gdf.to_crs(epsg=4326)
gdf['lon'] = gdf.geometry.centroid.x
gdf['lat'] = gdf.geometry.centroid.y

# Keep only essential columns to reduce memory load
gdf = gdf[['OBJECTID', 'lon', 'lat']]

# 2. Attach centroid coordinates to df
df = df.merge(gdf, on='OBJECTID')
df['geometry'] = gdf['OBJECTID']  # Only for completeness, not used in plotting

# 3. Approximate substation coordinates (average location of footprints per substation)
substation_coords = df.groupby('Substation Name GTET 100 Max Voltage')[['lon', 'lat']].mean().reset_index()
substation_coords.columns = ['Substation Name GTET 100 Max Voltage', 'sub_lon', 'sub_lat']

# 4. Merge back substation coordinates
df = df.merge(substation_coords, on='Substation Name GTET 100 Max Voltage', how='left')

# OPTIONAL: Sample only a subset to improve performance (e.g., top 500)
df_sampled = df.sample(n=500, random_state=42)

# 5. Generate connection lines from solar site → substation
lat_lines = []
lon_lines = []

for idx, row in df_sampled.iterrows():
    try:
        lon_lines.extend([row['lon'], row['sub_lon'], None])  # None breaks the line
        lat_lines.extend([row['lat'], row['sub_lat'], None])
    except:
        continue

# 6. Create Plotly map with footprints, substations, and connections
fig = go.Figure()

# Add solar footprint points
fig.add_trace(go.Scattermapbox(
    lat=df_sampled['lat'],
    lon=df_sampled['lon'],
    mode='markers',
    marker=dict(size=5, color='blue'),
    text=df_sampled['Substation Name GTET 100 Max Voltage'],
    name='Solar Footprints'
))

# Add substation (approximate) points
fig.add_trace(go.Scattermapbox(
    lat=substation_coords['sub_lat'],
    lon=substation_coords['sub_lon'],
    mode='markers',
    marker=dict(size=9, color='orange'),
    text=substation_coords['Substation Name GTET 100 Max Voltage'],
    name='Substations (approx.)'
))

# Add connection lines in a single trace
fig.add_trace(go.Scattermapbox(
    lat=lat_lines,
    lon=lon_lines,
    mode='lines',
    line=dict(width=1, color='gray'),
    showlegend=False
))

# 7. Update map layout
fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        zoom=5,
        center=dict(lat=df_sampled['lat'].mean(), lon=df_sampled['lon'].mean())
    ),
    title='Solar Footprints Connected to Nearest Substations (GTET 100kV)',
    margin=dict(t=50, b=10, l=10, r=10)
)

# 8. Save as interactive HTML file to avoid Jupyter crashes
fig.write_html("solar_to_GTET100kV_connections_optimized.html")
